# Linguistically-Aware EWC for Multilingual Continual Learning

**Research Question**: Does explicit linguistic similarity improve parameter preservation in multilingual continual learning?

## Experiment Setup
- **Languages**: Bengali → Hindi
- **Task**: Sentiment Analysis
- **Dataset**: ai4bharat/IndicSentiment
- **Model**: XLM-RoBERTa-base
- **Methods**: Naive, Standard EWC, Linguistic EWC, Random EWC

## 1. Setup Environment

In [ ]:
# Install dependencies
!pip install -q torch transformers datasets scikit-learn matplotlib seaborn tqdm

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Clone Repository (or upload files)

In [ ]:
# Option 1: Clone from GitHub (if you've pushed the code)
# !git clone https://github.com/yourusername/multilingual-cl-ling-ewc.git
# %cd multilingual-cl-ling-ewc

# Option 2: Upload files manually
# - Click folder icon on left
# - Upload: config.py, dataset.py, ewc.py, evaluation.py, train.py, visualize.py

# Verify files are present
!ls -la *.py

## 3. Test Data Loading

In [ ]:
# Test loading IndicSentiment dataset
from datasets import load_dataset

print("Loading Bengali dataset...")
dataset_bn = load_dataset("ai4bharat/IndicSentiment", "bn", split="train", trust_remote_code=True)
print(f"Bengali samples: {len(dataset_bn)}")

print("\nLoading Hindi dataset...")
dataset_hi = load_dataset("ai4bharat/IndicSentiment", "hi", split="train", trust_remote_code=True)
print(f"Hindi samples: {len(dataset_hi)}")

# Show sample
print("\n" + "="*60)
print("Sample Bengali review:")
print("="*60)
sample = dataset_bn[0]
print(f"Text: {sample.get('INDIC REVIEW', 'N/A')[:200]}...")
print(f"Label: {sample.get('LABEL', 'N/A')}")

print("\n" + "="*60)
print("Sample Hindi review:")
print("="*60)
sample = dataset_hi[0]
print(f"Text: {sample.get('INDIC REVIEW', 'N/A')[:200]}...")
print(f"Label: {sample.get('LABEL', 'N/A')}")

## 4. Configure Experiment

In [ ]:
from config import ExperimentConfig

# Create configuration
config = ExperimentConfig(
    model_name="xlm-roberta-base",
    num_epochs_per_language=3,
    batch_size=16,
    learning_rate=2e-5,
    ewc_lambda=5000.0,
    fisher_sample_size=1000,
    bengali_hindi_similarity=0.6,
    train_size=2000,  # Adjust based on available data
    eval_size=400,
    random_seed=42,
    output_dir="./results"
)

print("Experiment Configuration:")
print("="*60)
print(f"Model: {config.model_name}")
print(f"Epochs per language: {config.num_epochs_per_language}")
print(f"Batch size: {config.batch_size}")
print(f"EWC lambda: {config.ewc_lambda}")
print(f"Bengali-Hindi similarity: {config.bengali_hindi_similarity}")
print(f"Train size: {config.train_size}")
print(f"Methods: {config.methods}")
print("="*60)

## 5. Run Experiment

This will train 4 different methods:
1. **Naive**: No continual learning (baseline)
2. **Standard EWC**: Fixed penalty
3. **Linguistic EWC**: Similarity-scaled penalty (our method)
4. **Random EWC**: Random scaling (sanity check)

**Expected runtime**: ~30-60 minutes on GPU (T4), longer on CPU

In [ ]:
# Run the main experiment
!python train.py

## 6. Generate Visualizations

In [ ]:
!python visualize.py

## 7. View Results

In [ ]:
import json
from IPython.display import Image, display
import matplotlib.pyplot as plt

# Load comparison results
with open('./results/comparison.json', 'r') as f:
    comparison = json.load(f)

print("="*70)
print("EXPERIMENTAL RESULTS")
print("="*70)

for method in ['naive', 'ewc', 'ling_ewc', 'random_ewc']:
    if method in comparison:
        summary = comparison[method]
        print(f"\n{method.upper().replace('_', ' ')}:")
        print(f"  Average Forgetting: {summary['average_forgetting']:.4f}")
        print(f"  Final Avg Accuracy: {summary['average_final_accuracy']:.4f}")
        print(f"  Bengali Final: {summary['final_performance'].get('bengali', 0):.4f}")
        print(f"  Hindi Final: {summary['final_performance'].get('hindi', 0):.4f}")

# Show improvement
if 'ling_ewc_improvement_over_ewc' in comparison:
    improvement = comparison['ling_ewc_improvement_over_ewc']
    print("\n" + "="*70)
    print("KEY FINDING")
    print("="*70)
    print(f"Linguistic EWC vs Standard EWC:")
    print(f"  Reduction in forgetting: {improvement:.4f}")
    
    if improvement > 0.01:
        print(f"\n  ✅ RESULT: Linguistic scaling REDUCES forgetting by {improvement:.2%}")
        print(f"     → Linguistic structure matters for continual learning!")
    elif improvement < -0.01:
        print(f"\n  ❌ RESULT: Linguistic scaling INCREASES forgetting by {abs(improvement):.2%}")
        print(f"     → May need to adjust scaling strategy")
    else:
        print(f"\n  ➖ RESULT: No significant difference")
        print(f"     → Fisher Information may already capture linguistic structure")

print("\n" + "="*70)

In [ ]:
# Display visualizations
print("\n📊 Forgetting Comparison:")
display(Image('./results/forgetting_comparison.png'))

print("\n📊 Final Accuracy:")
display(Image('./results/final_accuracy.png'))

print("\n📊 Learning Curves:")
display(Image('./results/learning_curves.png'))

## 8. Download Results

In [ ]:
# Zip results for download
!zip -r results.zip ./results/

print("Results zipped! Download 'results.zip' from the files panel.")
print("\nContents:")
print("  - comparison.json (numerical results)")
print("  - *_metrics.json (per-method detailed metrics)")
print("  - *.png (visualization plots)")

## 9. Statistical Analysis (Optional)

In [ ]:
# For publishable results, run multiple seeds
import numpy as np

print("To get statistically significant results:")
print("1. Run experiment with multiple seeds (5-10 runs)")
print("2. Compute mean ± std for all metrics")
print("3. Run t-test to check significance")
print("\nExample code:")
print("""
for seed in [42, 43, 44, 45, 46]:
    config.random_seed = seed
    config.output_dir = f'./results_seed_{seed}'
    # Run experiment
    ...
""")

## Notes

### Interpretation Guide

**If Linguistic EWC reduces forgetting more than Standard EWC:**
- ✅ Your hypothesis is supported
- ✅ Linguistic similarity helps continual learning
- ✅ Ready for publication (after multiple seeds)

**If no significant difference:**
- Still valuable! Negative results are publishable
- Shows Fisher Information is linguistically-aware
- Suggests explicit bias unnecessary

### Next Steps

1. **More languages**: Add Marathi, Tamil for 3-4 language sequence
2. **Distance pairs**: Test Hindi→Tamil (distant) vs Hindi→Marathi (close)
3. **Different tasks**: Try NER, POS tagging
4. **Learned similarity**: Meta-learn optimal scaling
5. **Statistical rigor**: Multiple seeds, significance tests

### Citation

If this helps your research, cite:
- Kirkpatrick et al. (2017) - Original EWC
- AI4Bharat IndicSentiment dataset
- Your paper (once published!)